In [1]:
# Import Statements
import pandas as pd
import numpy as np
import difflib

pd.set_option('display.max_rows', None)

### Brewery Data

In [2]:
# Load Data
breweries = pd.read_csv('Data/wa_breweries.csv')

In [3]:
# View Data
breweries.head(5)

,brewery,satellite_location,brewery_type,address,city,zipcode,county,latitude,longitude,year_established,guild_member,production_data_2017,production_data_2022,closed_since_2022,untappd_profile_link
0,192 Brewing Company,Mount Vernon,taproom,1405 South 2nd Street,Mount Vernon,98273,Skagit,48.411774,-122.337970,2010,1,1,1,1,https://untappd.com/192BrewingCo
1,192 Brewing Company,NaN,brewpub,7324 NE 175th Street Ste F,Kenmore,98028,King,47.756677,-122.242502,2010,1,1,1,0,https://untappd.com/192BrewingCo
2,20 Corners Brewing Company,NaN,brewpub,14148 NE 190th St - Suite A,Woodinville,98072,King,47.765067,-122.151015,2016,0,1,1,0,https://untappd.com/20CornersBrewingCompany
3,210 Brewing Company,NaN,taproom,3438 Stoluckquamish Lane,Arlington,98223,Snohomish,48.213532,-122.184854,2015,1,1,0,0,https://untappd.com/210Brewing
4,23rd Ave Brewery,NaN,micro,2313 S Jackson St,Seattle,98144,King,47.599270,-122.301800,2018,0,0,1,0,https://untappd.com/23rdAveBrewery


In [4]:
# Data Info
breweries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   brewery               472 non-null    object 
 1   satellite_location    55 non-null     object 
 2   brewery_type          472 non-null    object 
 3   address               472 non-null    object 
 4   city                  472 non-null    object 
 5   zipcode               472 non-null    int64  
 6   county                472 non-null    object 
 7   latitude              472 non-null    float64
 8   longitude             472 non-null    float64
 9   year_established      472 non-null    int64  
 10  guild_member          472 non-null    int64  
 11  production_data_2017  472 non-null    int64  
 12  production_data_2022  472 non-null    int64  
 13  closed_since_2022     472 non-null    int64  
 14  untappd_profile_link  472 non-null    object 
dtypes: float64(2), int64(6)

In [5]:
#breweries['brewery'].unique()

### Production Data

In [6]:
# Method for cleaning production data
def clean_prod_df(df):
    
    # rename columns
    df = df.rename(columns={'Brewery':'brewery',
        '1':'January',    '2':'February', '3':'March',     '4':'April',
        '5':'May',        '6':'June',     '7':'July',      '8':'August',
        '9':'September', '10':'October', '11':'November', '12':'December'})
    
    # reorder columns
    df = df[['brewery','January','February','March','April','May','June',
             'July','August','September','October','November','December']]
    
    return df

In [7]:
# Method for consistent brewery names
def update_names(df):
    
    # remove periods
    df['brewery'] = df['brewery'].str.replace('.','')
    
    # replace end 'co' with 'company'
    df['brewery'] = np.where(df['brewery'].str[-2:] == 'CO',
                             df['brewery'].str[:-2] + 'COMPANY',
                             df['brewery'])

    # replace '&'' and 'and'
    df['brewery'] = df['brewery'].str.replace('&','AND')
    
    return df

In [8]:
# Method for getting summed annual production
def get_annual(df):
    df['annual_production'] = df.iloc[:,1:].sum(axis=1)
    return df

#### 2022 Production Data

In [9]:
# Load Data
production_2022 = pd.read_csv('Data/Production/brewery_monthly_production_2022.csv')

In [10]:
# Clean Data
production_2022 = clean_prod_df(production_2022)
production_2022 = update_names(production_2022)

In [11]:
# Sum Months
production_2022 = get_annual(production_2022)

In [12]:
# View Data
production_2022.head(5)

,brewery,January,February,March,April,May,June,July,August,September,October,November,December,annual_production
0,ANHEUSER-BUSCH,82718.39,86605.20,98521.98,84711.58,99589.33,92468.25,109179.90,121777.66,92403.54,82934.26,70992.27,69647.65,1091550.01
1,MILLERCOORS,72612.34,62743.02,77260.51,81442.02,93336.98,86396.11,84828.92,92874.04,88021.50,64324.32,73882.35,52628.00,930350.11
2,MARK ANTHONY BRANDS,22059.64,24801.43,39612.35,29996.00,42147.28,35631.05,23886.75,24572.94,31030.51,33271.36,19960.46,9499.35,336469.12
3,PABST BREWING,14641.35,12214.09,12939.65,9765.34,31595.86,19194.70,18157.79,17224.35,18875.10,15312.95,12107.57,13345.76,195374.51
4,GEORGETOWN BREWING COMPANY,6409.97,6503.01,8538.23,8672.18,8855.93,9571.99,8722.62,10193.51,9227.20,8985.20,9315.17,7380.14,102375.15


In [13]:
# Data Info
production_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brewery            865 non-null    object 
 1   January            510 non-null    float64
 2   February           552 non-null    float64
 3   March              536 non-null    float64
 4   April              531 non-null    float64
 5   May                536 non-null    float64
 6   June               539 non-null    float64
 7   July               543 non-null    float64
 8   August             543 non-null    float64
 9   September          562 non-null    float64
 10  October            528 non-null    float64
 11  November           574 non-null    float64
 12  December           532 non-null    float64
 13  annual_production  865 non-null    float64
dtypes: float64(13), object(1)
memory usage: 94.7+ KB


### Name Match Table

In [14]:
# function for matching names
def match_names(prod_data, brewery_data, cut=0.90):    

    # brewery names to test from
    names = []
    for n in brewery_data['brewery'].unique():
        n2 = n.lower().replace(' company','').replace(' brewery',' brewing')
        names.append((n,n2))

    # match to production data names
    prod_name_match = []
    for name in prod_data['brewery'].unique():
        name_test = name.lower().replace(' company','').replace(' inc','').replace(
            ' brewery',' brewing').replace(' -','').replace(' taproom','')
        matches = difflib.get_close_matches(name_test, [n[1] for n in names], cutoff=cut)
        if len(matches) > 0:
            m = matches[0]
            best_match = [n[0] for n in names if n[1] == m][0]
        else:
            best_match = None
        prod_name_match.append((name, best_match))
        
    return pd.DataFrame(prod_name_match, columns =['brewery_production', 'brewery_name'])

In [15]:
# 2022 production matching
prod_name_match_22 = match_names(production_2022, breweries)

### Merge on Name Match Table

In [16]:
cols = ['brewery','brewery_name','January','February','March','April','May', 'June',
        'July','August','September','October','November','December','annual_production']

In [17]:
# 2022 data
breweries_2022 = production_2022.merge(prod_name_match_22, how='inner', 
                                       left_on='brewery', right_on='brewery_production')
breweries_2022 = breweries_2022[cols]

In [18]:
# check non-matched breweries
non_matched = breweries_2022[breweries_2022['brewery_name'].isna()]['brewery'].unique()
#print(non_matched)

In [19]:
# get locations of non-matched breweries
brewery_locations = pd.read_csv('Data/brewery_locations.csv', index_col=0)
brewery_locations['state'] = brewery_locations['location'].apply(
    lambda x: None if type(x) != str else (x.split(', ')[1].split()[0] if len(x.split(', ')) > 1 
                                           and len(x.split(', ')[1].split()[0]) == 2 else None))

state_breweries = brewery_locations.groupby('state')['brewery'].apply(list)

In [20]:
# do not include breweries not in Washington
brews_to_remove = brewery_locations[brewery_locations['state'] != 'WA']['brewery'].tolist()

# add cider/kombucha/hop distributor places to removal list, MCMENAMINS EDGEFIELD is in OR
brews_to_remove.extend(['COMMUNITEA KOMBUCHA','KINE KOMBUCHA','PEAR UP CIDER','NEIGEL VINTNERS',
                        'TIMBER CITY GINGER BEER','YAKIMA VALLEY HOPS','MCMENAMINS EDGEFIELD'])

breweries_2022 = breweries_2022[~breweries_2022['brewery'].isin(brews_to_remove)]
breweries_2022.reset_index()

,index,brewery,brewery_name,January,February,March,April,May,June,July,August,September,October,November,December,annual_production
0,4,GEORGETOWN BREWING COMPANY,Georgetown Brewing Company,6409.97,6503.01,8538.23,8672.18,8855.93,9571.99,8722.62,10193.51,9227.20,8985.20,9315.17,7380.14,102375.15
1,10,FREMONT BREWING,Fremont Brewing Company,2466.78,2144.60,2800.91,2536.74,2616.72,3077.69,2720.60,2731.87,2819.80,2050.42,2435.67,3229.78,31631.58
2,12,MAC AND JACKS BREWERY INC,Mac and Jack's Brewing Company,1951.90,1841.37,1780.41,2036.64,2171.85,2776.15,2052.96,2412.73,2378.33,2225.78,1902.20,1755.14,25285.46
3,13,REUBENS BREWS,Reuben's Brews,1705.85,1676.63,1933.15,2070.59,2077.57,2519.17,2303.87,2617.09,2433.18,1740.98,1709.33,1592.80,24380.21
4,14,BALE BREAKER BREWING,Bale Breaker Brewing Company,1535.06,1276.72,1822.97,1900.88,1802.26,2140.11,2302.83,2282.84,1709.75,1536.51,2088.51,1416.19,21814.63
5,17,SILVER CITY BREWERY,Silver City Brewery,1248.72,1516.41,101.71,2372.86,2621.80,2218.78,1417.50,2169.09,1722.27,1524.62,1617.34,1591.01,20122.11
6,18,SCHOONER EXACT BREWING,Schooner Exact Brewing Company,-37.89,2567.50,2582.51,848.41,1477.33,1873.63,-170.53,6889.21,-70.33,1843.76,513.11,712.39,19029.10
7,22,NO-LI BREWHOUSE,No-Li Brewhouse,742.06,954.51,1113.38,1408.57,1056.81,1246.70,1396.96,1295.50,1409.41,1226.51,1029.92,1344.53,14224.86
8,24,ELYSIAN BREWING COMPANY,Elysian Brewing,874.69,780.65,-101.03,2182.40,1447.50,947.41,1211.80,1545.98,1207.30,1627.50,507.17,732.50,12963.87
9,25,IRON HORSE BREWERY,Iron Horse Brewery,998.40,1430.10,1344.42,1093.89,955.27,1146.92,1135.27,1237.49,1070.83,967.29,-41.19,1979.11,13317.80


In [21]:
# fix WA breweries
brews_to_match = brewery_locations[brewery_locations['state'] == 'WA']

wa_brews_fix = {
    '45 DEGREE BREWHOUSE':'45 Degree Brewhouse',
    'ANOTHER ROUND BREWING COMPANY':'Another Round Brewing Company',
    'AT LARGE BREWING AND':'At Large Brewing',
    'BARDIC BREWING':'Bardic Brewing and Cider',
    'BARLOWS BREWERY':"Barlow's Brewery",
    'CAPTAINS BREWING COMPANY':"Captain's Brewing Company",
    'CHIEF SPRINGS FIRE AND IRONS':"Chief Spring's Fire and Irons Brewpub",
    'COUNTERBALANCE BREWING':'Counterbalance Brewing Company',
    'DARACH BREWING COMPANY':'Darach Brewing Company',
    'DECIBEL BREWING COMPANY':'Decibel Brewing Company',
    'DUNAGAN BREWING COMPANY':'Dunagan Irish Pub and Brewery',
    'ENCHANTMENT BREWING':'Rock and Reef Brewing',
    'FATHOM AND LEAGUE HOP YARD':'Fathom and League Hop Yard Brewery',
    'FLYERS RESTAURANT AND':'Flyers Restaurant and Brewery',
    'FOB BREWING COMPANY':'Forward Operating Base Brewing Company',
    'HERE TODAY BREWERY AND':'Here Today Brewery and Kitchen',
    'HIGH BLUFF BREWING':'High Bluff Brewing',
    'HILDEGARD FERMENTS AND':'Hildegard Ferments and Botanicals',
    'HILLBILLY BREWING COMPANY':'Hillbilly Brewing Company',
    'LOGSDON FARMHOUSE ALES':'Logsdon Farmhouse Ales',
    'MAGNUSON BREWERY':'Magnuson Cafe and Brewery',
    'MATCHLESS':'Matchless Brewing',
    'MILLWOOD BREWING COMPANY':'Millwood Brewing Company',
    'PIKE BREWING COMPANY AND':'Pike Brewing Company',
    'PINT SIZE BREWING':'Pint Size Brewing',
    'POWERHOUSE RESTAURANT AND':'Powerhouse Restaurant and Brewery',
    'PRECIOUS THINGS':'Precious Things Fermentation Project',
    'RATTLESNAKE MOUNTAIN':'Rattlesnake Mountain Brewing Company',
    'ROSE GARAGE':'Rose Garage Brewing and Taps',
    'SADDLE ROCK PUB':'Saddle Rock Pub and Brewery',
    'SCHOONER EXACT BREWING':'Schooner Exact Brewing Company',
    'SLAUGHTER COUNTY BREWING':'Slaughter County Brewing Company',
    'SPOKANITE BREWING COMPANY':'Spokanite Brewing',
    'SQUIRREL FIGHT ARTISAN':'Squirrel Fight Artisan Brewing',
    'TERRAMAR':'Terramar Brewing and Distilling',
    'THE GOOD SOCIETY':'The Good Society Brewery',
    'TOP KNOT BREWING':'Top Knot Brewing',
    'TRUSTY BREWING COMPANY':'Trusty Brewing Company',
    'VASHON BREWING LLC':'Vashon Brewing',
    'VICTOR 23 BREWING':'Victor 23 Craft Brewery',
    'WANDERING HOP':'Wandering Hop Brewery',
    'WENATCHEE BREWING COMPANY':'Wenatchee Valley Brewing Company',
    'WILDLAND COOPERATIVE':'Wildland Cooperative',
    'YAKIMA CRAFT BREWING COMPANY':'Yakima Craft Brewing Company',
    'RAM BREWERY':'RAM Restaurant and Brewery',
    'SAN JUAN BREWING COMPANY':'San Juan Island Brewing Company',
    '54-40 BREWING COMPANY':"54°40' Brewing Company",
    'BAINBRIDGE ISLAND BREWING':'Bainbridge Brewing Company',
    'MEATHEADS SMOKEHOUSE AND':'Meatheads Smokehouse and Beer Works',
    'SOUND2SUMMIT BREWERY':'Sound To Summit Brewing',
    'FAST FASHION':'Fast Fashion Beer',
    'HALES ALES LTD':"Hale's Ales",
    'FLYING BIKE COOPERATIVE':'Flying Bike Cooperative Brewery',
    'WELL 80 ARTESIAN BREWING':'Well 80 Brewing Company',
    'NORTHWEST BREWERY WORKS':'Northwest Brewing Company',
    'MCMENAMINS':'McMenamins',
    'MCMENAMINS KALAMA HARBOR':'McMenamins Kalama Harbor Lodge',
    'MCMENAMINS ELKS TEMPLE':'McMenamins Elks Temple',
    'MCMENAMINS ANDERSON SCHOOL':'McMenamins Anderson School Brewery',
    'PENN COVE TAPROOM -':'Penn Cove Brewing Company',
    'PENN COVE BREWING CO LLC':'Penn Cove Brewing Company',
    'PENN COVE TAPROOM':'Penn Cove Brewing Company',
    '5 RIGHTS BREWING LLC':'5 Rights Brewing Company',
    'ALE SPIKE':'Ale Spike Camano Island Brewing',
    'ECHOES BREWING COMPANY LLC':'Echoes Brewing Company',
    'FOB BREWING COMPANY':'For the Love of God Brewing',
    'GENUS BREWING AND SUPPLY':'Genus Brewing',
    'NORTH FORK BARREL HOUSE AND':'North Fork Brewery',
    'NORTHWOOD PUBLIC HOUSE AND':'Northwood Public House and Brewery',
    "TTS OLD IRON BREWERY AND BBQ":"TT's Old Iron Brewery"}

for k, v in wa_brews_fix.items():
    breweries_2022['brewery_name'] = np.where(breweries_2022['brewery']==k, v, breweries_2022['brewery_name'])

In [22]:
# handle non-matched names
breweries_2022['brewery_name'] = np.where(breweries_2022['brewery_name'].isna(), 
                                          breweries_2022['brewery'],
                                          breweries_2022['brewery_name'])

# drop brewery column
breweries_2022.drop('brewery', axis=1, inplace=True)

In [23]:
# merge back with brewery info
cols = ['brewery_name','satellite_location','brewery_type',
        'address','city','zipcode','county','latitude','longitude',
        'year_established','guild_member','closed_since_2022','untappd_profile_link',
        'January','February','March','April','May', 'June','July','August',
        'September','October','November','December','annual_production']

brewery_production_2022 = breweries_2022.merge(breweries, how='left', left_on='brewery_name', right_on='brewery')
brewery_production_2022 = brewery_production_2022[cols]

In [24]:
brewery_production_2022.head()

,brewery_name,satellite_location,brewery_type,address,city,zipcode,county,latitude,longitude,year_established,...,April,May,June,July,August,September,October,November,December,annual_production
0,Georgetown Brewing Company,NaN,micro,5200 Denver Ave S,Seattle,98108.0,King,47.555251,-122.325538,2003.0,...,8672.18,8855.93,9571.99,8722.62,10193.51,9227.20,8985.20,9315.17,7380.14,102375.15
1,Fremont Brewing Company,NaN,micro,3409 Woodland Pk Ave N,Seattle,98103.0,King,47.648890,-122.344161,2009.0,...,2536.74,2616.72,3077.69,2720.60,2731.87,2819.80,2050.42,2435.67,3229.78,31631.58
2,Mac and Jack's Brewing Company,NaN,micro,17825 NE 65th St,Redmond,98052.0,King,47.663696,-122.102815,1993.0,...,2036.64,2171.85,2776.15,2052.96,2412.73,2378.33,2225.78,1902.20,1755.14,25285.46
3,Reuben's Brews,Ballard Taproom,regional,5010 14th Ave Nw,Seattle,98107.0,King,47.665402,-122.373301,2012.0,...,2070.59,2077.57,2519.17,2303.87,2617.09,2433.18,1740.98,1709.33,1592.80,24380.21
4,Reuben's Brews,Production Brewery,micro,800 NW 46th St,Seattle,98107.0,King,47.662412,-122.366598,2012.0,...,2070.59,2077.57,2519.17,2303.87,2617.09,2433.18,1740.98,1709.33,1592.80,24380.21


In [25]:
brewery_production_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   brewery_name          431 non-null    object 
 1   satellite_location    55 non-null     object 
 2   brewery_type          430 non-null    object 
 3   address               430 non-null    object 
 4   city                  430 non-null    object 
 5   zipcode               430 non-null    float64
 6   county                430 non-null    object 
 7   latitude              430 non-null    float64
 8   longitude             430 non-null    float64
 9   year_established      430 non-null    float64
 10  guild_member          430 non-null    float64
 11  closed_since_2022     430 non-null    float64
 12  untappd_profile_link  430 non-null    object 
 13  January               331 non-null    float64
 14  February              355 non-null    float64
 15  March                 3

In [26]:
# save csv
brewery_production_2022.to_csv('Data/brewery_production_2022.csv', index=False) 